In [41]:
import os
import pandas as pd 
import numpy as np
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

print(os.getcwd())
# DATA_PATH = "/bohr/PreEnzy-up6e/v1/data/"
DATA_PATH = "/bohr/ai4s-06g0/v1/Enzyme_v1/data"
raw_data = pd.read_csv(f"{DATA_PATH}/train.csv") 
test_data = pd.read_csv(f"{DATA_PATH}/test.csv") 

In [42]:
# Amino acids single-letter abbreviations ordered as in the image provided by the user
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Create a mapping from amino acid to index
aa_to_index = {aa: idx for idx, aa in enumerate(amino_acids)}

mutation_positions = [42, 43, 73, 74, 101, 143, 148, 173, 176, 177, 182]

In [43]:
# encode mutations into 1
def encode_mutation(mutation, updated_vector):
    # extract information from the `mutation`
    position = int(mutation[1:-1])
    new_aa = mutation[-1]
    
    position_index = mutation_positions.index(position) * len(amino_acids)
    amino_acid_index = aa_to_index[new_aa]
    updated_vector[position_index + amino_acid_index] = 1
    
    return updated_vector

def encode_mutations(mutations, initialized_vector):
    individual_mutations = mutations.split(';')
    updated_vector = initialized_vector
    for mutation in individual_mutations:
        # WHY assign the updated vector by the initialized vecotor
        # in each loop is also corrected here?
        # updated_vector = initialized_vector
        updated_vector = encode_mutation(mutation, updated_vector)
    return updated_vector

def generate_matrix(data):
    matrix = []
    for line in data.index:
        # Initialize the combined sparse vector for all mutations in the sequence
        initialized_vector = np.zeros(len(amino_acids) * len(mutation_positions), dtype=int)
        
        mutations = data['Mutant'][line]
        if mutations == "WT":
            matrix.append(initialized_vector)
            continue
        
        updated_vector = initialized_vector
        updated_vector = encode_mutations(mutations, updated_vector)
        matrix.append(updated_vector)
    matrix = np.array(matrix)
    return matrix

In [44]:
train_matrix = generate_matrix(raw_data)
test_matrix = generate_matrix(test_data)
train_matrix.shape, test_matrix.shape
# print(train_matrix[1])

In [45]:
# weighted mutations
def encode_mutation(mutation, updated_vector, weights_dict):
    # Extract information from the `mutation`
    position = int(mutation[1:-1])
    new_aa = mutation[-1]
    
    # Find the index for the mutation position
    position_index = mutation_positions.index(position)
    
    # Find the index for the new amino acid
    amino_acid_index = aa_to_index[new_aa]
    
    # Retrieve the weight for this mutation
    weight = weights_dict[position][new_aa]
    
    # Update the vector with the weighted value
    vector_index = position_index * len(amino_acids) + amino_acid_index
    updated_vector[vector_index] = weight
    # print(updated_vector)
    return updated_vector

def encode_mutations(mutations, initialized_vector, weights_dict):
    individual_mutations = mutations.split(';')
    updated_vector = initialized_vector
    for mutation in individual_mutations:
        # WHY assign the updated vector by the initialized vecotor
        # in each loop is also corrected here?
        # updated_vector = initialized_vector
        updated_vector = encode_mutation(mutation, updated_vector, weights_dict)
    return updated_vector

def generate_matrix(data, weights_dict):
    matrix = []
    for line in data.index:
        # Initialize the combined sparse vector for all mutations in the sequence
        initialized_vector = np.zeros(len(amino_acids) * len(mutation_positions), dtype=float)
        
        mutations = data['Mutant'][line]
        if mutations == "WT":
            matrix.append(initialized_vector)
            continue
        
        updated_vector = initialized_vector
        updated_vector = encode_mutations(mutations, updated_vector, weights_dict)
        matrix.append(updated_vector)
    matrix = np.array(matrix)
    return matrix

In [46]:
# Load the ligandmpnn weight matrix
weights_ligandmpnn_file = pd.read_csv("/bohr/ai4s-06g0/v1/Enzyme_v1/data/ligandmpnn.csv", index_col=0)
# print(weights_ligandmpnn_file)
# weights_ligandmpnn = weights_ligandmpnn_file.to_numpy()
# print(weights_ligandmpnn)

weights_ligandmpnn = {}

# Process each row in the DataFrame
for aa, row in weights_ligandmpnn_file.iterrows():
    for position, score in row.items():
        pos = int(position[1:])  # Extract the position as an integer
        if pos not in weights_ligandmpnn:
            weights_ligandmpnn[pos] = {}
        weights_ligandmpnn[pos][aa] = score

# print(weights_ligandmpnn)

In [47]:
! pip install openpyxl -i https://pypi.tuna.tsinghua.edu.cn/simple

In [48]:
# Load the ddG weight matrix
file_path = "/bohr/ai4s-06g0/v1/Enzyme_v1/data/ddG_b_f.xlsx"  # Update the file path to the actual file location
weights_ddg_file = pd.read_excel(file_path, header=None)

weights_ddg = {}

# Process each row in the DataFrame
for index, row in weights_ddg_file.iterrows():
    parts = row.dropna().tolist()  # Drop any NaN values and convert to list
    # print(parts)
    position = int(parts[0][1:])
    # print(position)
    original_residue = parts[0][0]
    # print(original_residue)
    
    # Initialize the dictionary for this position if not already done
    if position not in weights_ddg:
        weights_ddg[position] = {}
    
    # Find the original residue's score
    original_residue_index = parts.index(original_residue) + 1
    original_residue_score = float(parts[original_residue_index])
    
    # Process the mutations and their weights
    for i in range(1, len(parts), 2):
        mutant = parts[i]
        score = float(parts[i + 1])
        corrected_score = score - original_residue_score
        weights_ddg[position][mutant] = corrected_score

# print(weights_ddg)

In [49]:
train_matrix_weighted_ligandmpnn = generate_matrix(raw_data, weights_ligandmpnn)
test_matrix_weighted_ligandmpnn = generate_matrix(test_data, weights_ligandmpnn)
train_matrix_weighted_ligandmpnn.shape, test_matrix_weighted_ligandmpnn.shape
# print(train_matrix_weighted_ligandmpnn[1])

In [50]:
train_matrix_weighted_ddg = generate_matrix(raw_data, weights_ddg)
test_matrix_weighted_ddg = generate_matrix(test_data, weights_ddg)
train_matrix_weighted_ddg.shape, test_matrix_weighted_ddg.shape
# print(train_matrix_weighted_ddg[1])

In [51]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_predict
! pip install xgboost -i https://pypi.tuna.tsinghua.edu.cn/simple
from xgboost import XGBRegressor

In [52]:
seed = 2024
X = train_matrix_weighted_ddg
X_test = test_matrix_weighted_ddg
Y_train_activity = raw_data['Activity']
Y_train_selectivity = raw_data['Selectivity']

In [53]:
ridge_grid = [
    {
        'model': [Ridge()],
        'model__alpha': [0.1, 1.0, 10.0],
        'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'model__tol': [1e-2, 1e-3],
    }
]
pls_grid = [
    {
        'model': [PLSRegression()],
        'model__n_components': [2, 4, 6],
        'model__scale': [True, False],
        'model__max_iter': [1000],
        'model__tol': [1e-2, 1e-3],
    }
]

svm_grid = [
    {
        'model': [SVR()],
        'model__C' : [0.1, 1.0, 10.0],
        'model__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__degree' : [3],
        'model__gamma': ['scale'],
    }
]

rfr_grid = [
    {
        'model': [RandomForestRegressor()],
        'model__n_estimators' : [20],
        'model__criterion' : ['squared_error', 'absolute_error'],
        'model__max_features': ['sqrt', 'log2'],
        'model__min_samples_split' : [5, 10],
        'model__min_samples_leaf': [1, 4]
    }
]
xgb_grid= [
    {
        'model': [XGBRegressor()],
        'model__learning_rate': [0.01, 0.1], 
        'model__n_estimators': [100, 200], 
        'model__max_depth': [3, 5], 
        'model__min_child_weight': [1, 3],  
        'model__gamma': [0.0, 0.1], 
        'model__subsample': [0.8], 
        'model__colsample_bytree': [0.8], 
        'model__reg_alpha': [0.0],  
        'model__reg_lambda': [0.1], 
        'model__nthread': [4], 
    }
]

In [54]:
cls_list = [Ridge, PLSRegression, SVR, RandomForestRegressor, XGBRegressor]
param_grid_list = [ridge_grid, pls_grid, svm_grid, rfr_grid, xgb_grid]

pipe = Pipeline([
    ('model', 'passthrough')
])

def spearman_correlation(y_true, y_pred):
    return spearmanr(y_true, y_pred)[0]

spearman_scorer = make_scorer(spearman_correlation, greater_is_better=True)


def perform_grid_search(X_train, Y_train, score, cv = 10):
    result_list = []
    grid_list = []
    for cls_name, param_grid in zip(cls_list, param_grid_list):
        print(cls_name)
        grid = GridSearchCV(
            estimator = pipe,
            param_grid = param_grid,
            scoring = score,
            verbose = 1,
            n_jobs = -1,  # use all available cores
            cv = cv
        )
        grid.fit(X_train, Y_train)
        result_list.append(pd.DataFrame.from_dict(grid.cv_results_))
        grid_list.append(grid)
    return result_list, grid_list

result_list_activity, grid_list_activity = perform_grid_search(X, Y_train_activity, spearman_scorer)
result_list_selectivity, grid_list_selectivity = perform_grid_search(X, Y_train_selectivity, spearman_scorer)

def get_best_model(grid_list):
    best_score = -np.inf
    best_model = None
    best_index = -1
    for i, grid in enumerate(grid_list):
        if grid.best_score_ > best_score:
            best_score = grid.best_score_
            best_model = grid.best_estimator_
            best_index = i
    return best_model, best_index

# Get the best models for activity and selectivity
best_model_activity, best_index_activity = get_best_model(grid_list_activity)
best_model_selectivity, best_index_selectivity = get_best_model(grid_list_selectivity)

print(f"Best model for activity is at index: {best_index_activity}, model: {type(best_model_activity.named_steps['model'])}")
print(f"Best model for selectivity is at index: {best_index_selectivity}, model: {type(best_model_selectivity.named_steps['model'])}")

In [55]:
# Print the Spearman correlation score for the best models
def print_spearman_for_best_model(model, X, y, cv=10):
    preds = cross_val_predict(model, X, y, cv=cv)
    spearman_corr = spearmanr(y, preds)[0]
    print(f"Spearman correlation: {spearman_corr}")

print("Spearman correlation for best model (Activity):")
print_spearman_for_best_model(best_model_activity, X, Y_train_activity, cv=10)

print("Spearman correlation for best model (Selectivity):")
print_spearman_for_best_model(best_model_selectivity, X, Y_train_selectivity, cv=10)

In [56]:
# Predict Activity and Selectivity on the test set
test_activity_predictions = best_model_activity.predict(X_test)
test_selectivity_predictions = best_model_selectivity.predict(X_test)

test_predictions_df = pd.DataFrame({
    'Mutant': test_data['Mutant'],
    'Activity': test_activity_predictions,
    'Selectivity': test_selectivity_predictions
})
test_predictions_df.to_csv("./submission.csv", index=False) 